In [46]:
import numpy as np
import pandas as pd
import scipy as stats
%load_ext autoreload
%autoreload 2
from graph_implementation import Graph
from collections import defaultdict
import heapq as hq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sf_network  = pd.read_csv("sf_nodes_network.csv")
sf_edge_network = pd.read_csv("sf_edges_network.csv")

In [3]:
g = Graph()
# add all the road edges to the graph
for index, row in sf_edge_network.iterrows():
    start_edge = int(row["start_node_id"])
    end_edge = int(row["end_node_id"])
    g.addEdge(start_edge, end_edge)

In [16]:
for i in range(10000):
    start = np.random.randint(1,217000)
    end = np.random.randint(1,217000)
    s = g.isReachable(start, end)
    if s:
        print("True")

start node: 141196, end node: 157623
True
start node: 16043, end node: 16069
True
start node: 126685, end node: 134637
True
start node: 157707, end node: 164158
True
start node: 533, end node: 96661
True
start node: 50232, end node: 93257
True
start node: 65652, end node: 92462
True
start node: 7225, end node: 59874
True
start node: 44006, end node: 48093
True
start node: 19037, end node: 42157
True
start node: 21644, end node: 32716
True


In [17]:
# INITIALISER FOR USER POSITIONS 
for i in range(1000000):
    start = np.random.randint(1,174956)
    end = np.random.randint(1,174956)
    g.isReachable(start, end)

start node: 123964, end node: 136837
start node: 36139, end node: 58873
start node: 158553, end node: 166782
start node: 153778, end node: 153792
start node: 165810, end node: 165882
start node: 80945, end node: 105718
start node: 69394, end node: 96357
start node: 61806, end node: 84140
start node: 48100, end node: 124778
start node: 49385, end node: 99837
start node: 481, end node: 101578
start node: 13887, end node: 101328
start node: 32270, end node: 73447
start node: 128127, end node: 167677
start node: 140155, end node: 148738
start node: 144078, end node: 150472
start node: 8875, end node: 35907
start node: 70602, end node: 132847
start node: 49994, end node: 67259
start node: 68046, end node: 96689
start node: 169187, end node: 173570
start node: 19480, end node: 23572
start node: 40493, end node: 137288
start node: 60604, end node: 100123
start node: 35006, end node: 80806
start node: 55419, end node: 71877
start node: 86808, end node: 106591
start node: 82339, end node: 11097

start node: 49479, end node: 49492
start node: 1524, end node: 22061
start node: 75167, end node: 106898
start node: 36759, end node: 52172
start node: 9687, end node: 165150
start node: 27636, end node: 55140
start node: 136929, end node: 146626
start node: 102379, end node: 109218
start node: 171906, end node: 171911
start node: 54995, end node: 76121
start node: 153690, end node: 153690
start node: 69351, end node: 87548
start node: 83033, end node: 112610
start node: 128274, end node: 142962
start node: 32299, end node: 37644
start node: 44233, end node: 56510
start node: 53928, end node: 64705
start node: 152484, end node: 162103
start node: 102313, end node: 115847
start node: 17410, end node: 44829
start node: 8931, end node: 35955
start node: 32256, end node: 86682
start node: 28046, end node: 39867
start node: 126183, end node: 135105
start node: 8776, end node: 46436
start node: 126691, end node: 126724
start node: 45560, end node: 93244
start node: 34714, end node: 49304
sta

KeyboardInterrupt: 

In [90]:
def assignCarsToClosestUsers(currentUsersPosition, carsPos):
    
    if len(currentUsersPosition) == 0:
        return False
    newCarsPositions = dict()
    iter = 0
    for car in carsPos:
        minDist = 9999999999
        minUser = 0 
        carCoordinates = np.array([sf_network.loc[(sf_network["node_id"] == car)]["x_coord"].values[0], sf_network.loc[(sf_network["node_id"] == car)]["y_coord"].values[0]])
        for user in currentUsersPosition:
            #print("Current user: " + user)
            x_coor = sf_network.loc[(sf_network["node_id"] == int(user))]["x_coord"].values[0]
            y_coor = sf_network.loc[(sf_network["node_id"] == int(user))]["y_coord"].values[0]
            user_dist = np.array([x_coor, y_coor])
            l2_dist = np.linalg.norm(user_dist-carCoordinates) # choice of L2 distance is debatable
            temp_min_dist = min(minDist, l2_dist)
            if minDist != temp_min_dist: # i.e. if the L2 distance found is the new global minimum 
                minDist = temp_min_dist
                minUser = user
        currentUsersPosition.remove(minUser)
        newCarsPositions[car] = (minUser, minDist)
    return newCarsPositions, currentUsersPosition

After having run the "initialiser for user positions" code cell, the raw data was transferred to a .txt file before being converted to a CSV file in the following code cell:

In [20]:
import itertools
import csv
with open('acceptable_paths.txt', 'r') as input_file:
    stripped = (line.strip() for line in input_file)
    lines = (line.split(",") for line in stripped if line)
    with open('acceptable_paths.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('start_node', 'end_node'))
        writer.writerows(lines)

In [42]:
# After running the algorithm at the top of the page one million times, we obtain a certain number of exploitable paths
acceptable_paths = pd.read_csv("acceptable_paths.csv")
print("Number of acceptable paths found after one million calls: " + str(acceptable_paths.shape[0]))
acceptable_paths.head(5)

Number of acceptable paths found after one million calls: 436


,start_node,end_node
0,123964,136837
1,36139,58873
2,158553,166782
3,153778,153792
4,165810,165882


Now that we have our paths, let's initialise a certain number of users less than 436, say 300 to lower the odds that 2 people are assigned to the same initial node (by the Pigeonhole Principle), alhough this situation might frequently occur in real life as more people would likely book a car at busy places like the financial district, the train station or an airport. But in order to test the robustness of our use case, we need to have all our users at separate places in the graph. 

In [91]:
users = dict()
users_queue = []
for i in range(300):
    path_id = np.random.randint(0,436)
    start = str(acceptable_paths["start_node"][path_id])
    end = str(acceptable_paths["end_node"][path_id])
    hq.heappush(users_queue, start) # used to keep track of which users is still waiting to be picked up
    users[start] = end 

Now we can initialise the cars. We'll choose 10 cars, which is an order of magniture less than the number of users. 

In [48]:
cars = list()
for i in range(10):
    node_id = np.random.randint(0,174956)
    cars.append(node_id)

Now, let's assign users to each car.

In [92]:
carsPos = np.random.randint(1, 174956, size=10)
carPositions, remainingUsersPositions = assignCarsToClosestUsers(users_queue, carsPos)
print(carPositions)

{104931: ('117550', 262.0659590568874), 5194: ('91263', 886.6143927537943), 161922: ('156297', 30.874714813971043), 109789: ('64549', 560.7162056222517), 34232: ('32383', 49.00595215268676), 7538: ('6400', 77.84768876409616), 123019: ('91287', 383.6854455805479), 14321: ('8875', 65.97796629198632), 11828: ('24446', 726.3938863635126), 108520: ('95936', 195.6259178991826)}


The first element of the dictionary is the node ID of the car before the initialisation phase (before it's picked up its first user). The tuple includes the node ID of its first client as well as the normalised distance between the two.

In [93]:
print(len(remainingUsersPositions))

290


As expected, only 10 users have been picked up at the initialisation phase. We can now kick in the core of the algorithm.

In [108]:
g.bestFirstSearch(36139, 58873, sf_edge_network)

36139
36141
36142
36143
36144
36145
36147
36149
36146
36148
36151
36155
36158
36161
36165
36169
36171
36166
36180
36183
36181
36185
36187
36179
36175
36189
36150
36152
36153
36154
36157
36159
36162
36156
36164
36167
36172
36170
36174
36176
36178
36182
36184
36186
36188
36190
36192
36193
36194
36196
36197
36200
36198
36173
36177
36163
36191
36203
36206
36211
36218
36228
36238
36245
36240
36230
36220
36223
36236
36243
36246
36237
49666
44628
44637
44641
44644
44647
44648
44651
44643
44645
44646
36213
36217
36226
36229
36233
36234
36239
36227
49141
36224
36232
36241
36244
47905
48285
48286
44649
44653
36219
36205
36216
36160
36168
36195
36199
36201
36202
36207
36204
36208
36210
36212
36221
36209
36214
36225
36235
36222
36247
39583
39589
39594
44216
36231
36242
36249
39605
36215
39628
36248
45559
44495
44496
39616
39617
39618
39632
39640
39642
39643
39644
39645
39646
39647
39648
39650
39649
39651
39654
39655
39653
39657
39663
39658
39664
39667
39669
39659
39662
39668
39673
39677
58857
3965

[(6.435323, 58879),
 (10.149688000000001, 58880),
 (12.089308, 58881),
 (24.374771, 58876),
 (27.060284000000003, 58909),
 (16.274988, 58877),
 (14.371348000000001, 58874),
 (74.89155600000001, 39661)]

In [ ]:
    def runAlgorithm(self, carsPosition, remainingUsersPosition, acceptablePaths):
        remainingUsers = len(remainingUsersPosition)

        # initialise list of destination nodes for each car
        destinationNodes = dict()
        for car, userPos in carsPosition.items():
            destinationNodes[car] = acceptablePaths.loc[(acceptablePaths["start_node"] == int(userPos))]["end_node"].values[0]

        #initialise programmed paths for each car
        pathsPerCar = dict()
            for car, userPos in carsPosition.items():
                pathsPerCar[car] = g.bestFirstSearch(car, destinationNodes[car], sf_edge_network)
                
        # start the process
        currentStep = 0
        while remainingUsers > 0:
            currentCars = pathsPerCar.values()
            carPositionsAfterMovingOfOneNode = list()
            for lis in currentCars:
                carPositionsAfterMovingOfOneNode.append(currentCars[currentStep]) # get the node from the BFS-calculated path corresponding to
                                                                                 # the current step of the algorithm
            newCarPositions, newRemainingUsersPositions = assignCarsToClosestUsers(remainingUsersPosition, carPositionsAfterMovingOfOneNode)
            carsPosition = newCarPositions
            remainingUsersPosition = newRemainingUsersPositions
            initialPosition += 1 
                